# Run MinION with Guppy 

- This scripts runs guppy minion step by step. The actual run can be found in run_minion.py

In [1]:
# Import all packages

import sys
sys.path.append("/home/emre/github_repo/MinION")

from minION.util import IO_processor
from minION import analyser
from minION import consensus
from minION import demultiplexer
from minION.util import globals



import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import numpy as np
from Bio import SeqIO
import matplotlib.pyplot as plt
import gzip



In [2]:
# General settings

result_path = Path("/home/emre/")
experiment_name = "20230905_errorprone-3_test"

# Add conditions to avoid running the script accidentally
skip_basecalling = True
skip_demultiplex = False



### Step 1 (Optional): Basecall reads

- Basecall can usually be done while sequencing (if GPU available?)
- Otherwise, basecall afterwards

In [3]:
# Path to pod5 files
file_to_pod5 = "/var/lib/minknow/data/20230905_errorprone-3_test/no_sample/20230905_1342_MN41105_flg114_5c170bfa/pod5/"
basecall_model_type = "sup"
basecall_model = globals.DORADO_MODELS[basecall_model_type]
output_name = experiment_name


result_folder = IO_processor.create_folder( experiment_name,
                                            basecall_model_type, 
                                            target_path=result_path)
                                            
experiment_folder = IO_processor.find_experiment_folder(experiment_name) # Folder where pod5 files are located

basecall_folder = result_folder / "basecalled_filtered"
basecall_folder.mkdir(parents=True, exist_ok=True)

print(basecall_folder)
if not skip_basecalling:
    pod5_files = IO_processor.find_folder(experiment_folder, "pod5")
    run_dorado(basecall_model, pod5_files, basecall_folder, fastq = True)


/home/emre/minION_results/20230905_errorprone-3_test_sup/basecalled_filtered


### Step 2: Demultiplex Barcodes using guppy barcoder

- Define the barcodes in the barcoding folder and index them correctly
- We first run reverse barcodes and split in to # RBC folders. Within each RBC folder, guppy barcoder is run again for front barcodes


In [5]:
if not skip_demultiplex:

    BARCODES = {"Barcode-kit-FBC" : "FBC-v1",
                "Barcode-kit-RBC" : "RBC-v1"}

    output_folder_name = "demultiplex_50_guppy"

    demultiplexer.run_demultiplexer(result_folder, BARCODES, 50, 50, output_folder_name=output_folder_name, basecall_folder = basecall_folder)

Data_Path: /home/emre/github_repo/MinION/minION/barcoding
Barcode_rbc: RBC-v1 Barcode_fbc: FBC-v1
RBC Prompt: guppy_barcoder --input_path /home/emre/minION_results/20230905_errorprone-3_test_sup/basecalled_filtered --save_path /home/emre/minION_results/20230905_errorprone-3_test_sup/demultiplex_50_guppy --data_path /home/emre/github_repo/MinION/minION/barcoding --barcode_kits RBC-v1 --min_score_barcode_front 50
ONT Guppy barcoding software version 6.5.7+ca6d6af
input path:         /home/emre/minION_results/20230905_errorprone-3_test_sup/basecalled_filtered
save path:          /home/emre/minION_results/20230905_errorprone-3_test_sup/demultiplex_50_guppy
min. score front:   50
min. score rear:    50

Found 1 input files.

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
Done in 7573 ms.
/home/emre/minION_results/20230905_errorprone-3_test_sup/demultiplex_50_guppy
['/home/emre/min